#### LangChain vs LangGraph (feat. LangGraph 개념 설명)
* LangGraph의 개념과 주요 기능을 이해하고, 차이점을 비교합니다.

In [ ]:
# poetry add langgraph

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
print(UPSTAGE_API_KEY[30:])

sk
bc


In [2]:
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model='gpt-4o-mini') # 테스트의 경우에는 작은 모델을 사용합니다

from langchain_upstage import ChatUpstage
llm = ChatUpstage(
        model="solar-pro",
        base_url="https://api.upstage.ai/v1",
        temperature=0.5
    )
print(llm)

query = 'LangGraph는 무엇인가요?'
llm.invoke(query)



/Users/rose/Library/Caches/pypoetry/virtualenvs/mylangchain-app-75wTJiYg-py3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


client=<openai.resources.chat.completions.completions.Completions object at 0x135ad1550> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x135ad1fd0> model_name='solar-pro' temperature=0.5 model_kwargs={} upstage_api_key=SecretStr('**********') upstage_api_base='https://api.upstage.ai/v1'


AIMessage(content='**LangGraph**는 LLM(Large Language Model)을 활용한 복잡한 대화형 애플리케이션 구축을 위해 설계된 오픈소스 프레임워크로, **LangChain**의 확장성 있는 도구입니다. 주로 다음과 같은 특징을 가집니다:\n\n### 1. **핵심 개념**\n   - **상태 관리**: 사용자와의 대화 상태(History, Memory, Context)를 그래프 구조로 관리하여 장기적인 대화 흐름을 지원합니다.\n   - **그래프 기반 워크플로우**: 대화를 노드(Node)와 엣지(Edge)로 구성된 그래프로 모델링해 복잡한 논리적 흐름을 구현합니다 (예: 조건부 분기, 병렬 처리).\n   - **LLM 통합**: LangChain과 호환되어 다양한 LLM(GPT, Claude, Mistral 등)과 연동 가능합니다.\n\n### 2. **주요 사용 사례**\n   - **다단계 챗봇**: 사용자의 입력을 단계별로 처리하거나 외부 도구(검색, 데이터베이스 등)와 연동하는 애플리케이션.\n   - **에이전트 시스템**: 상태 기반 의사 결정이 필요한 작업 자동화 (예: 여행 계획 수립, 기술 지원).\n   - **대화형 인터페이스**: 사용자 입력을 분석해 동적으로 응답 경로를 변경하는 시스템.\n\n### 3. **LangChain과의 차이점**\n   - **LangChain**: 체이닝(Chain)과 메모리 관리에 중점을 둔 단순한 흐름 제어.\n   - **LangGraph**: 그래프 구조를 통해 더 유연한 상태 전이와 복잡한 대화 로직을 지원 (예: 상태 머신 패턴).\n\n### 4. **예시 아키텍처**\n   ```python\n   from langgraph import State, Graph\n\n   class ConversationState(State):\n       history: list[str] = []\n\n   graph = Graph(ConversationState)\

#### LangGraph의 기본개념
* `state`는 LangGraph 에이전트의 state를 나타내는 데이터 구조입니다.
* `state`는 `TypedDict`를 사용하여 정의되며, 이는 Python의 타입 힌팅을 통해 구조를 명확히 합니다.
    * 간단하게 `messages`라는 필드만 있습니다.
    * 필요에 따라 다양한 값들을 활용할 수 있습니다.
* `state`는 에이전트의 동작을 결정하는 데 사용되며, 각 노드에서 state를 업데이트하거나 참조할 수 있습니다.
* `state`는 LangGraph의 노드 간에 전달되며, 에이전트의 state 전이를 관리합니다.

In [3]:
from typing import Annotated # 타입 힌트를 사용하기 위해 
from typing_extensions import TypedDict # 구조화된 딕셔너리 타입을 정의하기 위해 

from langgraph.graph.message import add_messages 
from langchain_core.messages import AnyMessage # LangChain에서 사용하는 모든 종류의 메시지(예: HumanMessage, AIMessage)

# AgentState는 에이전트의 현재 상태를 나타내는 딕셔너리 타입을 정의합니다.
# TypedDict를 사용하면 딕셔너리가 어떤 키와 값 타입을 가져야 하는지 명확하게 지정할 수 있습니다.
class AgentState(TypedDict):
    # 'messages' 키는 에이전트의 대화 기록을 저장합니다.
    # 이 목록에는 LangChain 메시지 객체(AnyMessage)가 들어갑니다.
    # LangGraph가 이 상태를 처리할 때, 새로운 메시지가 추가되면
    # 기존 메시지 목록의 끝에 자동으로 추가되도록(append) 설정합니다.
    messages: list[Annotated[AnyMessage, add_messages]]

- 위에 선언한 `AgentState`를 활용하여 `StateGraph`를 생성합니다.

In [4]:
from langgraph.graph import StateGraph

graph_builder = StateGraph(AgentState)

- `graph`에 추가할 `node`를 생성합니다
-  `node`는 LangGraph에서 실행되는 개별적인 작업 단위를 의미합니다. 
    - 각 노드는 특정 기능을 수행하는 독립적인 컴포넌트로, 예를 들어 텍스트 생성, 데이터 처리, 또는 의사 결정과 같은 작업을 담당할 수 있습니다.
    - `node`는 기본적으로 함수(function)로 정의되고, 뒤에서 다루지만 다른 에이전트(agent)를 활용할 수도 있습니다

In [5]:
def generate(state: AgentState) -> AgentState:
    """
    `generate` 노드는 사용자의 질문을 받아서 응답을 생성하는 노드입니다.
    """
    messages = state['messages']
    ai_message = llm.invoke(messages)
    return {'messages': [ai_message]}

- `node`를 생성한 후에 `edge`로 연결합니다
- `edge`는 노드들 사이의 연결을 나타내며, 데이터와 제어 흐름의 경로를 정의합니다. 
    - 엣지를 통해 한 노드의 출력이 다음 노드의 입력으로 전달되어, 전체적인 워크플로우가 형성됩니다.
    - `node`와 `edge`의 조합은 방향성 그래프(Directed Graph)를 형성하며, 이를 통해 복잡한 AI 에이전트의 행동 흐름을 구조화할 수 있습니다

In [6]:
graph_builder.add_node('generate', generate)

- 모든 그래프는 `START(시작)`와 `END(종료)`가 있습니다
    - `END`를 explicit하게 선언하지 않는 경우도 종종 있지만, 가독성을 위해 작성해주는 것을 권장합니다

In [7]:
from langgraph.graph import START, END

graph_builder.add_edge(START, 'generate')
graph_builder.add_edge('generate', END)

- `node`를 생성하고 `edge`로 연결한 후에 `compile` 메서드를 호출하여 `Graph`를 생성합니다

In [23]:
graph = graph_builder.compile()
print(type(graph))

<class 'langgraph.graph.state.CompiledStateGraph'>


- `compile` 후에는 그래프를 시각화하여 확인할 수 있습니다
- 의도한대로 그래프가 생성됐는지 확인하는 습관을 기르는 것이 좋습니다
    - `git`에서 코드 작업물을 commit하기 전에 `git diff`를 통해 변경사항을 확인하는 것과 같습니다

In [ ]:
#from IPython.display import display, Image

#display(Image(graph.get_graph().draw_mermaid_png(max_retries=5, retry_delay=2.0)))

In [25]:
# 대체 방법
mermaid_code = graph.get_graph().draw_mermaid()
print("Mermaid Code:")
print(mermaid_code)

Mermaid Code:
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	generate(generate)
	__end__([<p>__end__</p>]):::last
	__start__ --> generate;
	generate --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



* https://mermaid.live/ 에서  mermain_code 로 직접 확인한다.


In [26]:
from langchain_core.messages import HumanMessage

initial_state = {'messages': [HumanMessage(query)]}
graph.invoke(initial_state)

{'messages': [AIMessage(content='**LangGraph**는 **LangChain**과 통합된 **그래프 기반의 상태 관리 프레임워크**로, 복잡한 LLM(Large Language Model) 애플리케이션의 흐름을 구조화하고 관리하기 위해 설계되었습니다. 주로 **대화형 멀티모달 애플리케이션**이나 **상태 유지가 필요한 장기 작업**에 적합하며, 다음과 같은 핵심 특징을 가집니다:\n\n---\n\n### 1. **주요 개념**\n- **그래프 구조**: 상태 전이를 노드와 간선으로 표현해 복잡한 대화 흐름을 시각화합니다.\n- **상태 관리**: 사용자 세션, 메모리, 외부 데이터베이스와의 연동을 체계적으로 처리합니다.\n- **LangChain 통합**: LangChain의 체인, 에이전트와 결합해 유연한 파이프라인 구축이 가능합니다.\n\n---\n\n### 2. **사용 사례**\n- **대화형 챗봇**: 사용자 입력을 기반으로 동적 대화 흐름 제어 (예: 폼 채우기, 의사결정 트리).\n- **장기 작업**: 여러 단계에 걸친 작업 처리 (예: 예약 시스템, 문서 분석).\n- **멀티모달 애플리케이션**: 텍스트, 이미지, 음성 등을 조합한 복합 작업.\n\n---\n\n### 3. **주요 구성 요소**\n- **Nodes**: LLM 호출, 도구 사용, 상태 업데이트 등의 단위 작업.\n- **Edges**: 노드 간 전환 조건 (예: 사용자 입력에 따른 분기).\n- **State**: 그래프 실행 중 유지되는 데이터 (메모리, 세션 정보 등).\n\n---\n\n### 4. **LangChain과의 차별점**\n- **LangChain**은 단일/순차적 체인에 강점이 있지만, **LangGraph**는 분기, 루프, 병렬 처리 등 **비선형 흐름**에 최적화되어 있습니다.\n- 예시:  \n  ```python\n  from langgraph import State, BaseGraph\n  from langchai